In [1]:
# importamos librerias varias
import pandas as pd
from multiprocessing import Pool
from pandas_ods_reader import read_ods
import glob

In [2]:
%%time
# Cargamos desde los pickles
salientes = pd.read_pickle('salientes.pkl')
entrantes = pd.read_pickle('entrantes.pkl')

CPU times: user 1.21 s, sys: 467 ms, total: 1.68 s
Wall time: 1.68 s


In [3]:
salientes.sample(20)

,Nro. Llamante,Destino,Nro. Llamado,fecha y hora,Duracion (Seg),Id. celda,Descripc. Celda,Latitud,Longitud,Dir Celda
64413,1159038416,Conexión Móvil,926612282,27/08/2023 12:47:55,1106,CF01212G100MB2,BRIGADA AÉREA,"-34,7002","-58,7373",CURAPALIGUE AL 1000 - MERLO - BUENOS AIRES
94761,1158789291,Conexión Móvil,926175715,26/08/2023 15:03:16,152,CF02081L100MA1,CRUCERO BELGRANO 2,"-34,722076","-58,724047",RP100(Av.Patricios) entre Murillo y Azul 1003
105938,1125776107,TRAFICO DATOS,290710217,27/08/2023 17:41:08,2502,CF02081L200MB3,CRUCERO BELGRANO 2,"-34,722049","-58,724124",RP100(Av.Patricios) entre Murillo y Azul 1003
124300,1125491787,Conexion Movil,22596023,27/08/2023 19:47:16,151184705,CF01212L100MB3,Brigada Aérea,"-34,700357","-58,737708",Curapaligue 1069
66639,1136981246,Tráfico incluid,328874103,22/08/2023 11:33:26,3805,CF05175L100MC1,CRUCERO BELGRANO 3,"-34,7042","-58,723560",
14454,1168110439,Conexión Móvil,272108848,27/08/2023 2:49:23,633,CF05175U100MB2,Crucero Belgrano 3,"-34,703730","-58,724950",BEBEDERO 1992
103110,1158786976,Conexión Móvil,947635963,09/09/2023 16:01:04,304,CF01148L100MC1,LIBERTAD 2,"-34,7097","-58,7097",PEREYRA 50
40570,1132298688,Conexión Móvil,325680805,04/09/2023 9:29:02,13734,CF05175U100MC1,Crucero Belgrano 3,"-34,703730","-58,724950",BEBEDERO 1992
157425,1157172637,LLAM. LOCAL,1156958577,11/09/2023 23:51:55,3,CF01147U100MA1,CRUCERO BELGRANO,"-34,709811","-58,732226",Avda Mosconi 1815
159573,2205001231,Tráfico incluid,916409757,13/08/2023 18:30:40,32943,CF01212L100MA3,Brigada Aérea,"-34,700357","-58,737708",Curapaligue 1069


In [4]:
# sacamos un listado de telefonos y la cantidad de cada uno
resumen_llamantes = salientes['Nro. Llamante'].value_counts()
resumen_llamantes

1151467918    79226
1145635562    70662
1165409227    48861
1133294894    45722
1121765241    42413
              ...  
2291501888        1
1154181869        1
1176115172        1
1158764385        1
1121804490        1
Name: Nro. Llamante, Length: 69414, dtype: int64

In [7]:
# Contamos la cantidad de numeros de telefonos unicos
salientes['Nro. Llamante'].nunique()

69414

In [16]:
salientes.dtypes

Nro. Llamante      datetime64[ns]
Destino                    object
Nro. Llamado                int64
fecha y hora               object
Duracion (Seg)              int64
Id. celda                  object
Descripc. Celda            object
Latitud                    object
Longitud                   object
Dir Celda                  object
fecha y  hora      datetime64[ns]
dtype: object

In [14]:
salientes['fecha y  hora'] = pd.to_datetime(salientes['fecha y hora'])


In [17]:
salientes.sample(15)

,Nro. Llamante,Destino,Nro. Llamado,fecha y hora,Duracion (Seg),Id. celda,Descripc. Celda,Latitud,Longitud,Dir Celda,fecha y hora
148383,1970-01-01 00:00:01.162214644,Tráfico incluid,335376989,26/08/2023 20:20:48,232988,CF01147L100MC1,CRUCERO BELGRANO,"-34,710134","-58,731501",Avda Mosconi 1815,2023-08-26 20:20:48
51226,1970-01-01 00:00:01.157560630,TRAFICO DATOS,260971297,08/08/2023 9:27:10,7988,CF02081L200MA3,CRUCERO BELGRANO 2,"-34,722049","-58,724124",RP100(Av.Patricios) entre Murillo y Azul 1003,2023-08-08 09:27:10
41623,1970-01-01 00:00:01.178997114,Conexión Móvil,501730034,19/08/2023 9:23:55,2045,CF02081U100MC1,CRUCERO BELGRANO 2,"-34,722076","-58,724047",RP100(Av.Patricios) entre Murillo y Azul 1003,2023-08-19 09:23:55
72756,1970-01-01 00:00:01.138831912,Tráfico incluid,198396690,26/08/2023 12:58:27,16272,CF01212L100MC1,BRIGADA AÉREA,"-34,700240","-58,737350",Curapaligue 1069,2023-08-26 12:58:27
154440,1970-01-01 00:00:01.132720968,Conexión Móvil,21307560,12/08/2023 19:59:07,1020,CF01148G100MB2,LIBERTAD 2,"-34,7043","-58,7043",LIBERTAD 2,2023-12-08 19:59:07
138364,1970-01-01 00:00:01.121919426,Tráfico incluid,970848423,25/08/2023 19:08:46,998,CF02081L200MA3,CRUCERO BELGRANO 2,"-34,722049","-58,724124",RP100(Av.Patricios) entre Murillo y Azul 1003,2023-08-25 19:08:46
71776,1970-01-01 00:00:01.157681101,Tráfico incluid,501004133,19/08/2023 12:46:59,139904,CF01212L100MC1,BRIGADA AÉREA,"-34,700240","-58,737350",Curapaligue 1069,2023-08-19 12:46:59
94915,1970-01-01 00:00:01.132542025,TRAFICO DATOS,267542955,19/08/2023 15:10:34,128,CF05175L100MA3,CRUCERO BELGRANO 3,"-34,7042","-58,723560",,2023-08-19 15:10:34
67709,1970-01-01 00:00:01.132053249,LLAM. LOCAL,1156613347,12/09/2023 11:33:55,58,CF01212U100MA2,BRIGADA AÉREA,"-34,700240","-58,737350",Curapaligue 1069,2023-12-09 11:33:55
4217,1970-01-01 00:00:01.176151175,Conexión Móvil,317066040,28/08/2023 0:50:38,222,CF01212G100MA2,BRIGADA AÉREA,"-34,7002","-58,7373",CURAPALIGUE AL 1000 - MERLO - BUENOS AIRES,2023-08-28 00:50:38
